In [1]:
import h2o
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

In [2]:
h2o.init()

H2O cluster uptime:,25 minutes 52 seconds 461 milliseconds
H2O cluster version:,3.3.0.99999
H2O cluster name:,anqi_fu
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [27]:
from h2o.h2o import _locate # private function. used to find files within h2o git project directory.

# Import walking gait data
gait = h2o.upload_file(path=_locate("smalldata/glrm_test/subject01_walk1.csv"))
gait.describe()


Parse Progress: [##################################################] 100%
Uploaded py2d1f12d0-bc6f-457b-a8e6-13944d3628ae into cluster with 151 rows and 124 cols
Rows: 151 Cols: 124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.19:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0



Column-by-Column Summary:



,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.6,107.79163,600.01556,1028.7,-152.5,399.40009,1034.9,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.3,550.69415,684.75494,-133.588,435.01773,627.49988,-170.9,319.34705,444.43289,157.69023,376.44724,403.25574,102.54192,293.89526,368.28577,165.6133,341.2023,438.80435,-173.0,365.72482,375.95242,-121.0,304.46347,393.67728,-173.4,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,58.7473,-74.0361,211.85968,69.02702,-69.30095,185.39088,48.22646,-140.1,254.42706,36.65646,-88.18266,627.38177,1342.9,-35.20795,511.80032,1427.4,164.48206,551.72424,1453.6,-249.9,485.94559,1227.9,216.05853,531.12134,1217.6,-266.6,424.32767,1129.1,242.29697,460.03223,1123.8,-304.8,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.6,502.82971,886.58942,-300.9,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,199.80693,36.02512,-153.1,228.04898,42.9706,-54.88903,675.97009,1562.3,44.15432,683.50897,1567.3,-94.07465,614.13971,1740.4,-19.48838
maxs,2.5,641.12048,1091.4,180.07411,661.46918,1078.0,-80.37424,451.5914,1077.8,55.36993,692.89514,788.11365,217.07512,766.44531,750.72375,136.25021,728.67242,672.42242,209.89946,674.53168,783.71576,-138.1,773.97705,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.5,821.66901,425.57733,-63.51654,750.77588,473.02411,-124.5,812.50183,320.70309,108.161,935.61414,198.72064,86.30818,934.44775,211.37376,188.70589,1073.9,179.28133,153.59363,821.7641,321.80249,-34.92287,971.31763,178.00603,-6.83138,955.73096,184.36398,-10

In [46]:
# Plot first row of data on x- vs. y-coordinate features
gait_row = gait[1,:].drop("Time")
gait_row_np = np.array(h2o.as_list(gait_row))
x_coords = range(0, gait_row_np.shape[1], 3)
y_coords = range(1, gait_row_np.shape[1], 3)

x_pts = gait_row_np[0,x_coords]
y_pts = gait_row_np[0,y_coords]
plt.plot(x_pts, y_pts, 'bo')

# Add feature labels to each point
feat_names = [nam[:-2] for nam in gait_row.col_names[1::3]]
for i in xrange(len(feat_names)):
    plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
plt.title("Location of Body Parts at Time 0")
plt.xlabel("X-Coordinate Weight")
plt.ylabel("Y-Coordinate Weight")
plt.show()

In [4]:
# Basic GLRM using quadratic loss and no regularization (PCA)
model = h2o.glrm(x=gait[1:], k=10, loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=1000)
model.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2ODimReductionModel :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445394997134_11

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,14.0,0.0,369810.7




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 369810.748004
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-20 20:02:32,0.082 sec,0.0,1.05,371905.1
,2015-10-20 20:02:32,0.091 sec,1.0,0.7,371905.1
,2015-10-20 20:02:32,0.100 sec,2.0,0.5,371905.1
,2015-10-20 20:02:32,0.109 sec,3.0,0.3,371905.1
,2015-10-20 20:02:32,0.118 sec,4.0,0.2,371905.1
,2015-10-20 20:02:32,0.127 sec,5.0,0.1,371905.1
,2015-10-20 20:02:32,0.136 sec,6.0,0.0,371905.1
,2015-10-20 20:02:32,0.146 sec,7.0,0.0,371905.1
,2015-10-20 20:02:32,0.155 sec,8.0,0.0,371905.1
,2015-10-20 20:02:32,0.165 sec,9.0,0.0,369810.7


In [5]:
# Plot objective function value each iteration
model_score = model.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
plt.plot(model_score["iteration"], model_score["objective"])
plt.show()

In [20]:
# Archetype to feature mapping (Y)
gait_y = model._model_json["output"]["archetypes"]
print gait_y

gait_y_np = np.array(model.archetypes())
x_coords = range(0, gait_y_np.shape[1], 3)
y_coords = range(1, gait_y_np.shape[1], 3)

# Plot archetypes on x- vs. y-coordinate features
for k in xrange(gait_y_np.shape[0]):
    x_pts = gait_y_np[k, x_coords]
    y_pts = gait_y_np[k, y_coords]
    plt.plot(x_pts, y_pts, 'bo')

    # Add feature labels to each point
    feat_names = [nam[:-1] for nam in gait_y.col_header[1::3]]
    for i in xrange(len(feat_names)):
        plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
    plt.title("Feature Weights of Archetype " + str(k+1))
    plt.xlabel("X-Coordinate Weight")
    plt.ylabel("Y-Coordinate Weight")
    plt.show()


Archetypes:


,rasisx,rasisy,rasisz,lasisx,lasisy,lasisz,vsacralx,vsacraly,vsacralz,rthighupperx,rthighuppery,rthighupperz,rthighfrontx,rthighfronty,rthighfrontz,rthighrearx,rthighreary,rthighrearz,lthighupperx,lthighuppery,lthighupperz,lthighfrontx,lthighfronty,lthighfrontz,lthighrearx,lthighreary,lthighrearz,rshankupperx,rshankuppery,rshankupperz,rshankfrontx,rshankfronty,rshankfrontz,rshankrearx,rshankreary,rshankrearz,lshankupperx,lshankuppery,lshankupperz,lshankfrontx,lshankfronty,lshankfrontz,lshankrearx,lshankreary,lshankrearz,rheelx,rheely,rheelz,rmidfootsupx,rmidfootsupy,rmidfootsupz,rmidfootlatx,rmidfootlaty,rmidfootlatz,rtoetipx,rtoetipy,rtoetipz,lheelx,lheely,lheelz,lmidfootsupx,lmidfootsupy,lmidfootsupz,lmidfootlatx,lmidfootlaty,lmidfootlatz,ltoetipx,ltoetipy,ltoetipz,sternumx,sternumy,sternumz,racromiumx,racromiumy,racromiumz,lacromiumx,lacromiumy,lacromiumz,rbicepx,rbicepy,rbicepz,lbicepx,lbicepy,lbicepz,relbowx,relbowy,relbowz,lelbowx,lelbowy,lelbowz,rwristmedx,rwristmedy,rwristmedz,rwristlatx,rwristlaty,rwristlatz,lwristmedx,lwristmedy,lwristmedz,lwristlatx,lwristlaty,lwristlatz,rtoelatx,rtoelaty,rtoelatz,rtoemedx,rtoemedy,rtoemedz,ltoelatx,ltoelaty,ltoelatz,ltoemedx,ltoemedy,ltoemedz,rtemplex,rtempley,rtemplez,ltemplex,ltempley,ltemplez,topheadx,topheady,topheadz
Arch1,621.4,1056.9,153.4,644.7,1051.2,-106.6,434.3,1052.8,12.4,544.6,737.9,205.4,610.5,664.5,126.9,521.6,619.0,201.6,657.5,749.7,-157.3,753.8,740.3,-91.5,689.6,646.8,-140.6,402.5,486.4,190.8,442.8,408.7,143.2,359.0,412.6,194.4,727.5,441.5,-133.7,779.2,398.8,-77.4,713.2,396.8,-142.1,115.8,226.8,98.1,224.4,139.4,74.5,196.8,130.5,177.7,297.5,43.3,118.4,682.5,73.9,-48.5,848.2,85.0,-17.9,835.5,77.3,-115.4,961.6,71.8,-45.0,657.3,1360.4,-6.2,557.6,1458.3,199.6,590.1,1464.2,-218.8,575.4,1259.4,253.5,592.9,1225.3,-243.2,539.8,1155.9,289.9,536.4,1132.5,-277.1,707.6,946.8,276.9,673.5,928.9,329.9,635.8,901.2,-231.6,587.8,891.6,-289.3,237.2,58.7,195.4,262.9,59.3,86.6,898.6,56.6,-122.6,912.2,66.2,-20.3,723.7,1583.3,75.1,737.6,1584.9,-63.3,733.4,1759.2,11.8
Arch2,611.4,1076.5,176.8,614.0,1071.3,-84.3,413.9,1073.2,52.4,567.7,752.7,215.4,627.3,691.3,121.0,556.6,630.8,202.3,643.5,774.8,-149.3,745.5,768.7,-94.2,683.6,672.2,-143.0,479.5,460.2,184.7,552.3,418.1,131.4,483.3,373.6,188.0,680.2,484.2,-141.6,710.7,422.0,-93.9,645.8,452.9,-145.6,373.4,74.7,93.0,516.8,79.6,65.4,504.9,65.0,171.9,657.8,48.8,113.0,488.7,181.7,-42.2,637.2,124.5,-35.6,609.5,124.3,-129.6,721.9,66.4,-75.2,640.1,1385.8,40.1,548.6,1469.6,256.8,558.9,1502.1,-161.8,576.7,1267.0,300.4,550.2,1265.7,-201.7,548.4,1160.1,332.1,486.8,1178.1,-238.3,739.3,974.6,298.2,708.9,948.1,349.6,583.9,943.7,-206.1,535.2,937.1,-263.2,591.2,40.1,189.8,602.0,47.2,80.3,653.7,80.5,-144.2,682.8,82.0,-45.0,697.7,1619.5,132.6,706.2,1627.3,-6.4,675.7,1793.0,74.9
Arch3,607.0,1085.4,125.5,620.4,1067.3,-134.3,414.9,1070.9,-1.4,648.6,773.7,194.3,734.8,729.9,114.2,679.2,655.8,195.7,568.0,762.5,-176.7,656.6,727.7,-108.6,566.8,653.5,-154.3,608.9,520.4,182.5,658.0,445.6,141.3,572.3,443.9,188.7,548.4,438.9,-142.4,592.1,388.9,-85.8,526.8,396.5,-148.1,354.2,243.8,81.5,470.4,162.0,77.6,433.3,158.1,176.5,550.7,56.9,139.0,438.0,75.7,-45.1,606.2,74.1,-15.4,593.8,65.0,-111.3,719.5,49.1,-41.6,634.5,1385.5,-20.5,520.8,1480.5,179.4,579.3,1488.8,-235.6,511.6,1283.8,234.0,622.7,1251.8,-251.9,458.3,1186.7,263.5,588.2,1149.1,-289.4,578.3,945.1,280.9,531.8,937.1,327.2,727.3,948.6,-220.8,690.7,927.7,-282.7,479.2,85.6,203.8,515.1,85.1,99.8,655.7,41.4,-118.3,668.8,49.8,-16.6,692.7,1614.5,59.1,709.4,1617.1,-79.1,691.5,1791.2,-5.3
Arch4,632.5,1067.7,154.4,622.2,1053.3,-106.6,429.9,1058.1,43.3,654.8,752.4,204.6,723.5,706.4,108.7,674.4,632.4,193.1,568.1,751.7,-168.5,668.6,714.9,-124.4,572.8,642.6,-161.1,640.3,457.7,180.4,715.5,424.2,124.0,655.9,372.4,185.5,502.1,466.9,-165.8,500.3,395.7,-114.6,454.5,454.0,-167.1,577.4,71.9,96.1,721.9,81.6,70.1,709.2,66.6,176.9,863.2,55.9,119.4,177.3,294.9,-58.0,274.1,149.0,-46.0,243.2,158.3,-134.9,309.0,55.1,-69.8,651.0,1368.5,34.0,555.0,1448.

In [6]:
# Projection into archetype space (X)
x_key = model._model_json["output"]["representation_name"]
gait_x = h2o.get_frame(x_key)
gait_x.show()

time_np = np.array(h2o.as_list(gait["Time"]))
gait_x_np = np.array(h2o.as_list(gait_x))

# Plot archetypes over time
lines = []
for i in xrange(gait_x_np.shape[1]):
    lines += plt.plot(time_np, gait_x_np[:,i], '-')
plt.title("Archetypes over Time")
plt.xlabel("Time")
plt.ylabel("Archetypal Projection")
plt.legend(lines, gait_x.col_names)
plt.show()

H2OFrame with 151 rows and 5 columns: 


,Arch1,Arch2,Arch3,Arch4,Arch5
0,1.391761,0.268824,-0.031825,-0.434455,-0.173159
1,1.373343,0.202596,0.016253,-0.362360,-0.211344
2,1.353799,0.136736,0.067958,-0.284862,-0.257593
3,1.320208,0.066878,0.116344,-0.196884,-0.292933
4,1.271825,-0.007750,0.163189,-0.095890,-0.319866
5,1.214632,-0.080229,0.207192,0.013402,-0.345302
6,1.129461,-0.156821,0.240841,0.140678,-0.346313
7,1.038164,-0.226880,0.268955,0.275939,-0.349912
8,0.941674,-0.284724,0.287521,0.414215,-0.353720
9,0.835542,-0.331202,0.293834,0.555850,-0.350384


In [7]:
# Reconstruct data from X and Y
pred = model.predict(gait)
pred.head()

[['reconstr_R.ASIS.X',
  'reconstr_R.ASIS.Y',
  'reconstr_R.ASIS.Z',
  'reconstr_L.ASIS.X',
  'reconstr_L.ASIS.Y',
  'reconstr_L.ASIS.Z',
  'reconstr_V.Sacral.X',
  'reconstr_V.Sacral.Y',
  'reconstr_V.Sacral.Z',
  'reconstr_R.Thigh.Upper.X',
  'reconstr_R.Thigh.Upper.Y',
  'reconstr_R.Thigh.Upper.Z',
  'reconstr_R.Thigh.Front.X',
  'reconstr_R.Thigh.Front.Y',
  'reconstr_R.Thigh.Front.Z',
  'reconstr_R.Thigh.Rear.X',
  'reconstr_R.Thigh.Rear.Y',
  'reconstr_R.Thigh.Rear.Z',
  'reconstr_L.Thigh.Upper.X',
  'reconstr_L.Thigh.Upper.Y',
  'reconstr_L.Thigh.Upper.Z',
  'reconstr_L.Thigh.Front.X',
  'reconstr_L.Thigh.Front.Y',
  'reconstr_L.Thigh.Front.Z',
  'reconstr_L.Thigh.Rear.X',
  'reconstr_L.Thigh.Rear.Y',
  'reconstr_L.Thigh.Rear.Z',
  'reconstr_R.Shank.Upper.X',
  'reconstr_R.Shank.Upper.Y',
  'reconstr_R.Shank.Upper.Z',
  'reconstr_R.Shank.Front.X',
  'reconstr_R.Shank.Front.Y',
  'reconstr_R.Shank.Front.Z',
  'reconstr_R.Shank.Rear.X',
  'reconstr_R.Shank.Rear.Y',
  'reconstr_R.S

In [8]:
# Plot original and reconstructed L.Acromium.X over time
lacro_np = np.array(gait["L.Acromium.X"])
lacro_pred_np = np.array(pred["reconstr_L.Acromium.X"])
line_orig = plt.plot(time_np, lacro_np, '-')
line_imp = plt.plot(time_np, lacro_pred_np, '-')

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
plt.legend([line_orig, line_imp], ["Original", "Imputed"])
plt.show()

In [47]:
# Import walking gait data with missing values
gait_miss = h2o.upload_file(_locate("smalldata/glrm_test/subject01_walk1_miss15.csv"))
gait_miss.describe()


Parse Progress: [##################################################] 100%
Uploaded py21c6d303-7fb4-4d0a-bcdd-a0419feb2a8f into cluster with 151 rows and 124 cols
Rows: 151 Cols: 124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.19:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0



Column-by-Column Summary:



,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.6,107.79163,600.01556,1029.3,-152.5,399.40009,1034.9,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.3,550.69415,684.75494,-133.588,435.01773,627.49988,-170.9,319.34705,445.08429,157.69023,376.44724,403.25574,102.54192,295.04089,368.36661,165.6133,341.2023,438.80435,-173.0,365.72482,375.95242,-121.0,304.46347,393.67728,-173.4,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,60.01719,-74.0361,211.85968,70.39124,-69.30095,185.39088,48.22646,-140.1,254.42706,36.65646,-88.18266,627.38177,1342.9,-35.20795,511.80032,1427.4,164.48206,551.72424,1453.6,-249.9,485.94559,1227.9,216.05853,531.12134,1217.6,-266.6,424.32767,1129.5,242.4415,460.20703,1123.8,-304.8,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.6,503.10611,886.58942,-300.9,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,202.74066,36.02512,-153.1,228.04898,42.9706,-54.88903,675.97009,1562.3,44.15432,683.50897,1567.3,-94.07465,614.13971,1740.4,-19.48838
maxs,2.5,641.12048,1091.3,180.07411,661.46918,1078.0,-80.82956,451.5914,1077.8,55.29718,692.89514,788.11365,216.86717,766.44531,750.72375,136.25021,726.57483,672.42242,209.89946,674.53168,781.90839,-138.1,772.35175,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.5,821.66901,425.57733,-63.825,750.77588,472.92294,-125.0268,812.50183,320.70309,108.161,935.61414,197.97754,85.62352,934.44775,209.43217,188.70589,1071.8,179.28133,153.59363,821.7641,321.80249,-34.92287,971.31763,178.00603,-6.83138,955.73096,184.36398,-1

In [49]:
# Basic GLRM using quadratic loss and no regularization
model2 = h2o.glrm(x=gait_miss[1:], validation_x=gait[1:], k=10, init="SVD", svd_method = "GramSVD", loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=2500, min_step_size=1e-6)
model2.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2ODimReductionModel :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445394997134_14

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,2349.0,0.0,122072.0




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 122071.962792
Misclassification Error (Categorical): 0.0

ModelMetricsGLRM: glrm
** Reported on validation data. **

MSE: NaN
Sum of Squared Error (Numeric): 153399.030972
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-20 20:26:08,0.462 sec,0.0,0.7,4709528.9
,2015-10-20 20:26:08,0.470 sec,1.0,0.4,4709528.9
,2015-10-20 20:26:08,0.479 sec,2.0,0.2,4709528.9
,2015-10-20 20:26:08,0.487 sec,3.0,0.1,4709528.9
,2015-10-20 20:26:08,0.495 sec,4.0,0.0,4709528.9
---,---,---,---,---,---
,2015-10-20 20:26:51,42.645 sec,2344.0,0.0,122072.1
,2015-10-20 20:26:51,42.673 sec,2345.0,0.0,122072.1
,2015-10-20 20:26:51,42.702 sec,2346.0,0.0,122072.0
,2015-10-20 20:26:51,42.731 sec,2347.0,0.0,122072.0


In [50]:
# Plot objective function value each iteration
model2_score = model2.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
plt.plot(model2_score["iteration"], model2_score["objective"])
plt.show()

In [51]:
# Impute missing data from X and Y
pred2 = model2.predict(gait_miss)
pred2.head()

H2OFrame with 151 rows and 123 columns: 


reconstr_R.ASIS.X,reconstr_R.ASIS.Y,reconstr_R.ASIS.Z,reconstr_L.ASIS.X,reconstr_L.ASIS.Y,reconstr_L.ASIS.Z,reconstr_V.Sacral.X,reconstr_V.Sacral.Y,reconstr_V.Sacral.Z,reconstr_R.Thigh.Upper.X,reconstr_R.Thigh.Upper.Y,reconstr_R.Thigh.Upper.Z,reconstr_R.Thigh.Front.X,reconstr_R.Thigh.Front.Y,reconstr_R.Thigh.Front.Z,reconstr_R.Thigh.Rear.X,reconstr_R.Thigh.Rear.Y,reconstr_R.Thigh.Rear.Z,reconstr_L.Thigh.Upper.X,reconstr_L.Thigh.Upper.Y,reconstr_L.Thigh.Upper.Z,reconstr_L.Thigh.Front.X,reconstr_L.Thigh.Front.Y,reconstr_L.Thigh.Front.Z,reconstr_L.Thigh.Rear.X,reconstr_L.Thigh.Rear.Y,reconstr_L.Thigh.Rear.Z,reconstr_R.Shank.Upper.X,reconstr_R.Shank.Upper.Y,reconstr_R.Shank.Upper.Z,reconstr_R.Shank.Front.X,reconstr_R.Shank.Front.Y,reconstr_R.Shank.Front.Z,reconstr_R.Shank.Rear.X,reconstr_R.Shank.Rear.Y,reconstr_R.Shank.Rear.Z,reconstr_L.Shank.Upper.X,reconstr_L.Shank.Upper.Y,reconstr_L.Shank.Upper.Z,reconstr_L.Shank.Front.X,reconstr_L.Shank.Front.Y,reconstr_L.Shank.Front.Z,reconstr_L.Shank.Rear.X,reconstr_L.Shank.Rear.Y,reconstr_L.Shank.Rear.Z,reconstr_R.Heel.X,reconstr_R.Heel.Y,reconstr_R.Heel.Z,reconstr_R.Midfoot.Sup.X,reconstr_R.Midfoot.Sup.Y,reconstr_R.Midfoot.Sup.Z,reconstr_R.Midfoot.Lat.X,reconstr_R.Midfoot.Lat.Y,reconstr_R.Midfoot.Lat.Z,reconstr_R.Toe.Tip.X,reconstr_R.Toe.Tip.Y,reconstr_R.Toe.Tip.Z,reconstr_L.Heel.X,reconstr_L.Heel.Y,reconstr_L.Heel.Z,reconstr_L.Midfoot.Sup.X,reconstr_L.Midfoot.Sup.Y,reconstr_L.Midfoot.Sup.Z,reconstr_L.Midfoot.Lat.X,reconstr_L.Midfoot.Lat.Y,reconstr_L.Midfoot.Lat.Z,reconstr_L.Toe.Tip.X,reconstr_L.Toe.Tip.Y,reconstr_L.Toe.Tip.Z,reconstr_Sternum.X,reconstr_Sternum.Y,reconstr_Sternum.Z,reconstr_R.Acromium.X,reconstr_R.Acromium.Y,reconstr_R.Acromium.Z,reconstr_L.Acromium.X,reconstr_L.Acromium.Y,reconstr_L.Acromium.Z,reconstr_R.Bicep.X,reconstr_R.Bicep.Y,reconstr_R.Bicep.Z,reconstr_L.Bicep.X,reconstr_L.Bicep.Y,reconstr_L.Bicep.Z,reconstr_R.Elbow.X,reconstr_R.Elbow.Y,reconstr_R.Elbow.Z,reconstr_L.Elbow.X,reconstr_L.Elbow.Y,reconstr_L.Elbow.Z,reconstr_R.Wrist.Med.X,reconstr_R.Wrist.Med.Y,reconstr_R.Wrist.Med.Z,reconstr_R.Wrist.Lat.X,reconstr_R.Wrist.Lat.Y,reconstr_R.Wrist.Lat.Z,reconstr_L.Wrist.Med.X,reconstr_L.Wrist.Med.Y,reconstr_L.Wrist.Med.Z,reconstr_L.Wrist.Lat.X,reconstr_L.Wrist.Lat.Y,reconstr_L.Wrist.Lat.Z,reconstr_R.Toe.Lat.X,reconstr_R.Toe.Lat.Y,reconstr_R.Toe.Lat.Z,reconstr_R.Toe.Med.X,reconstr_R.Toe.Med.Y,reconstr_R.Toe.Med.Z,reconstr_L.Toe.Lat.X,reconstr_L.Toe.Lat.Y,reconstr_L.Toe.Lat.Z,reconstr_L.Toe.Med.X,reconstr_L.Toe.Med.Y,reconstr_L.Toe.Med.Z,reconstr_R.Temple.X,reconstr_R.Temple.Y,reconstr_R.Temple.Z,reconstr_L.Temple.X,reconstr_L.Temple.Y,reconstr_L.Temple.Z,reconstr_Top.Head.X,reconstr_Top.Head.Y,reconstr_Top.Head.Z
615.6,1055.7,165.4,637.7,1045.1,-96.0,428.0,1054.5,23.0,512.7,741.9,206.0,564.3,666.0,117.1,474.6,626.1,193.9,652.0,744.6,-149.9,746.0,735.4,-80.6,688.6,641.7,-138.6,353.8,472.6,180.2,413.8,413.3,128.6,335.3,386.4,184.6,747.0,444.1,-133.9,812.4,415.0,-84.1,743.4,397.1,-141.9,154.5,108.4,95.9,297.5,86.9,63.1,284.3,70.1,170.8,427.1,43.8,107.5,811.7,69.6,-56.4,965.4,123.4,-32.6,947.3,112.7,-128.5,1074.1,139.0,-65.7,659.6,1359.3,9.8,566.9,1455.3,221.6,589.4,1469.0,-199.3,596.2,1254.8,271.4,574.9,1232.0,-230.3,570.2,1148.5,307.8,509.1,1143.0,-264.3,775.2,982.2,265.1,750.9,950.3,319.4,612.7,910.5,-225.1,563.2,900.8,-281.7,362.9,32.7,188.6,374.3,38.7,77.3,1015.7,110.5,-140.0,1030.0,122.8,-38.9,725.6,1590.5,91.7,734.1,1594.8,-48.2,714.1,1762.0,28.9
618.0,1054.9,163.8,640.8,1045.7,-98.3,430.5,1054.9,20.1,510.7,741.6,205.9,563.5,663.9,118.7,471.6,625.5,194.7,652.5,746.0,-151.7,746.9,736.5,-83.1,688.1,643.3,-139.0,348.9,473.5,182.0,407.7,412.3,131.5,328.4,388.0,186.9,744.3,443.9,-133.9,809.3,413.0,-83.6,739.9,396.7,-142.4,142.4,117.4,99.1,283.3,89.7,67.2,268.0,71.6,174.8,407.7,43.2,111.5,797.8,66.4,-55.6,955.1,114.4,-31.4,938.2,104.4,-127.3,1065.9,128.8,-62.7,662.5,1357.4,6.3,570.9,1455.2,218.0,594.0,1468.3,-203.3,597.2,1255.3,268.8,577.7,1230.9,-234.1,569.4,1149.2,306.3,511.

In [54]:
# Plot original and imputed L.Acromium.X over time
time_np = np.array(h2o.as_list(gait["Time"]))
lacro_np2 = np.array(h2o.as_list(gait["L.Acromium.X"]))
lacro_pred_np2 = np.array(h2o.as_list(pred2["reconstr_L.Acromium.X"]))
plt.plot(time_np, lacro_np2, 'b-')
plt.plot(time_np, lacro_pred_np2, 'g-')

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
blue_patch = mpatches.Patch(color = 'blue', label = 'Original')
green_patch = mpatches.Patch(color = 'green', label='Imputed')
plt.legend([blue_patch, green_patch], ["Original", "Imputed"])
plt.show()

Rows: 151 Cols: 124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.19:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0



Column-by-Column Summary:



,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.6,107.79163,600.01556,1029.3,-152.5,399.40009,1034.9,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.3,550.69415,684.75494,-133.588,435.01773,627.49988,-170.9,319.34705,445.08429,157.69023,376.44724,403.25574,102.54192,295.04089,368.36661,165.6133,341.2023,438.80435,-173.0,365.72482,375.95242,-121.0,304.46347,393.67728,-173.4,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,60.01719,-74.0361,211.85968,70.39124,-69.30095,185.39088,48.22646,-140.1,254.42706,36.65646,-88.18266,627.38177,1342.9,-35.20795,511.80032,1427.4,164.48206,551.72424,1453.6,-249.9,485.94559,1227.9,216.05853,531.12134,1217.6,-266.6,424.32767,1129.5,242.4415,460.20703,1123.8,-304.8,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.6,503.10611,886.58942,-300.9,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,202.74066,36.02512,-153.1,228.04898,42.9706,-54.88903,675.97009,1562.3,44.15432,683.50897,1567.3,-94.07465,614.13971,1740.4,-19.48838
maxs,2.5,641.12048,1091.3,180.07411,661.46918,1078.0,-80.82956,451.5914,1077.8,55.29718,692.89514,788.11365,216.86717,766.44531,750.72375,136.25021,726.57483,672.42242,209.89946,674.53168,781.90839,-138.1,772.35175,780.81445,-66.30186,713.16107,681.38623,-123.1844,719.69702,531.05676,200.56787,799.47015,457.5513,154.56749,751.88885,458.64368,202.59061,764.30676,494.35605,-116.5,821.66901,425.57733,-63.825,750.77588,472.92294,-125.0268,812.50183,320.70309,108.161,935.61414,197.97754,85.62352,934.44775,209.43217,188.70589,1071.8,179.28133,153.59363,821.7641,321.80249,-34.92287,971.31763,178.00603,-6.83138,955.73096,184.36398,-1